In [1]:
# 
# author: William G.K. Martin 
# email: willful_liam@gmail.com 
# copyright (c) 2013
# liscence: BSD style
# 

"""
Module with a generating samples from a multivariate Gaussian distribution. 




# Usage
draws = gaussian_draw(mean, covariance_matrix, draws=1)

## in
mean              ~ array shaped (N,)
covariance_matrix ~ array shaped (N, N)
n_draws           ~ number of samples generate.

## out
draws             ~ array shaped (n_draws, N)


# Test
The routine `gaussian_draw` is tested by looking at the Chi2 statistics of 
many samples that are generated for a (mean, cov_mat) pair defined below. Recall
that `E(chi2) = N` and `Var(chi2) = 2N`.


"""



# standard
from __future__ import print_function


# scipy
import scipy as sp
import scipy.linalg as la 
from scipy import stats
from scipy.optimize import fminbound, fmin_slsqp
import numpy.testing as nptest
    
from matplotlib.pyplot import figure
from time import time




# function definitions
# --

def gaussian_draw(mean, covariance_matrix, n_draws=1, draw=stats.norm.rvs):
    """
    Return a random sample of the given multivariate gaussian 
    density function defined by the mean and covariance matrix.

    kwds
    -----
    draws  - numer of smaples to make (returned along first dimention)

    """
    # check for diagonality of covariance matrix
    Sdiag = covariance_matrix.diagonal()
    frob = (covariance_matrix**2).sum() 
    frob_diag = (Sdiag**2).sum()
    Croot = la.cholesky(covariance_matrix).transpose()
    N = mean.size
    x_standards = draw(size=N*n_draws).reshape(N, n_draws)
    out = sp.dot(Croot, x_standards).T + mean
    return sp.squeeze(out)

def chi2_moments(x, S, mean=None):
    """
    Compute the normalized 
    S ~ (N,N)
    x ~ (Nsamples, N)
    """
    # shape testing
    # --
    Nsamples, N = x.shape[:]
    for _N in S.shape:
        assert N==_N
    
    DOF = N-1 if (id(mean) is id(None)) else N
    mean = x.mean(0) if (mean is None) else mean

    # compute the chi squared values
    LUpiv = la.lu_factor(S) 
    chi2 = sp.array([sp.dot((mean-_x), la.lu_solve(LUpiv, (mean-_x))) for _x in x])
    
    # The draw for probability S can be tested by 
    chi2_mean = chi2.mean()
    chi2_var = ((chi2-chi2_mean)**2).sum()/(Nsamples-1.0)
    std = sp.sqrt(chi2_var)
    chi2_skew = 1.0 / float(Nsamples) * ((chi2-chi2_mean)**3).sum() / std**3
    return DOF, chi2_mean, chi2_var, chi2_skew 


In [13]:

# Test routine for the gaussian_draw function

## Define a distribution with a certain covariance and mean
N = 4
x = sp.rand(N)
S = sp.eye(N) * (1 + .5 * sp.rand(N))[:, None]
S = S + .0005 * sp.rand(N, N)
S = sp.dot(S.T, S)

## number of samples    
Nsamples = 10000


# multiple call method
print("Computing the draws for the multivariate distribution with, \n")
print("mean={}\n".format(x))
print("S={}.\n\n".format(S))
print("==\n")

%time xdraws = sp.array([gaussian_draw(x, S) for i in range(Nsamples)])



print("\n==\n\nTest the chi2 statics of the draw:\n")

N, chi2_mean, chi2_var, chi2_skew = chi2_moments(xdraws, S, mean=x)    

print("N = {0}\nchi2_mean = {1}\nchi2_var = {2}\nchi2_skew = {3}\n".format(
        *chi2_moments(xdraws, S, mean=x)))


print("Theoretical chi2 statistices:")
print("chi2_mean = {}".format(N))
print("chi2_var  = {}".format(2 * N))
print("chi2_skew = {}".format(sp.sqrt(8 / float(N))))

Computing the draws for the multivariate distribution with, 

mean=[0.13940712 0.53181158 0.00475798 0.91243239]

S=[[1.23647978e+00 3.45798607e-04 3.32269188e-04 5.38466959e-04]
 [3.45798607e-04 1.05787114e+00 9.23826705e-04 2.97418466e-04]
 [3.32269188e-04 9.23826705e-04 1.60680042e+00 8.01070263e-04]
 [5.38466959e-04 2.97418466e-04 8.01070263e-04 1.00155391e+00]].


==

CPU times: user 1.04 s, sys: 20 ms, total: 1.06 s
Wall time: 1.05 s

==

Test the chi2 statics of the draw:

N = 4
chi2_mean = 4.047057811606494
chi2_var = 8.507550065387482
chi2_skew = 1.54660629602194

Theoretical chi2 statistices:
chi2_mean = 4
chi2_var  = 8
chi2_skew = 1.4142135623730951
